We will be testing our [AFL fuzzer](http://lcamtuf.coredump.cx/afl/) on LibreSSL 2.1.6. The link to the tutorial I am following can be found [here](https://necurity.co.uk/netsec/2015/03/30/Fun-With-AFL.html).

Going through the tutorial I encountered some issues. Since my account is not root, I decided to install LibreSSL to **_~/testing/_** instead of the specified account.

In setting up the library path for LD_LIBRARY_PATH, you must use the full path, and not the **_~_** shortcut for home. 

> **_export LD_LIBRARY_PATH="/home/willy/testing/usr/local/lib"_**

When I ran 

> **_~/testing/usr/local/bin/openssl req -x509 -nodes -days 365 -newkey rsa:512 -keyout privateKey.key -out certificate.crt -out CSR.csr_**

a certificate.crt file was not created. In order to create it I ran

> **_~/testing/usr/local/bin/openssl req -key privateKey.key -new -x509 -days 365 -out certificate.crt _**

Then, after making the directories in the tutorial, I moved each filetype to its respective folder for scanning.

In [1]:
!../afl-fuzz -i inkey -o outkey -- ~/testing/usr/local/bin/openssl x509 -in @@ -text -noout

afl-fuzz 1.83b by <lcamtuf@google.com>

[-] Oops, your terminal window seems to be smaller than 80 x 25 characters.
    That's not enough for afl-fuzz to correctly draw its fancy ANSI UI!

    Depending on the terminal software you are using, you should be able to
    resize the window by dragging its edges, or to adjust the dimensions in
    the settings menu.
)B
[-] PROGRAM ABORT : Please resize terminal to 80x25 or more
         Location : check_terminal(), afl-fuzz.c:6626



Since the above code doesn't work in here, I will simply copy and paste the results into here

Note that when attempting to run afl-fuzz we may encounter the following error:

[+] You have 4 CPU cores and 3 runnable tasks (utilization: 75%).
[+] Try parallel jobs - see /usr/local/share/doc/afl/parallel_fuzzing.txt.
[*] Checking core_pattern...
[*] Checking CPU scaling governor...

[-] Whoops, your system uses on-demand CPU frequency scaling, adjusted
    between 781 and 2344 MHz. Unfortunately, the scaling algorithm in the
    kernel is imperfect and can miss the short-lived processes spawned by
    afl-fuzz. To keep things moving, run these commands as root:

    cd /sys/devices/system/cpu
    echo performance | tee cpu*/cpufreq/scaling_governor

    You can later go back to the original state by replacing 'performance' with
    'ondemand'. If you don't want to change the settings, set AFL_SKIP_CPUFREQ
    to make afl-fuzz skip this check - but expect some performance drop.

[-] PROGRAM ABORT : Suboptimal CPU scaling governor
         Location : check_cpu_governor(), afl-fuzz.c:6925

To avoid this we simply run the given commands, using sudo for the tee command

So after letting it run all night on privatekeys nothing was found:


>                       american fuzzy lop 1.83b (openssl)
>
>┌─ process timing ─────────────────────────────────────┬─ overall results ─────┐

>│        run time : 0 days, 9 hrs, 13 min, 39 sec      │  cycles done : 2      │

>│   last new path : 0 days, 0 hrs, 4 min, 28 sec       │  total paths : 512    │

>│ last uniq crash : none seen yet                      │ uniq crashes : 0      │

>│  last uniq hang : none seen yet                      │   uniq hangs : 0      │

>├─ cycle progress ────────────────────┬─ map coverage ─┴───────────────────────┤

>│  now processing : 4* (0.78%)        │    map density : 2651 (4.05%)          │

>│ paths timed out : 0 (0.00%)         │ count coverage : 1.49 bits/tuple       │

>├─ stage progress ────────────────────┼─ findings in depth ────────────────────┤

>│  now trying : havoc                 │ favored paths : 88 (17.19%)            │

>│ stage execs : 4732/5000 (94.64%)    │  new edges on : 137 (26.76%)           │

>│ total execs : 4.97M                 │ total crashes : 0 (0 unique)           │

>│  exec speed : 149.8/sec             │   total hangs : 0 (0 unique)           │

>├─ fuzzing strategy yields ───────────┴───────────────┬─ path geometry ────────┤

>│   bit flips : 3/138k, 4/138k, 4/138k                │    levels : 5          │

>│  byte flips : 0/17.3k, 1/14.6k, 3/14.5k             │   pending : 395        │

>│ arithmetics : 5/816k, 0/88.1k, 0/9735               │  pend fav : 0          │

>│  known ints : 0/87.7k, 0/401k, 6/635k               │ own finds : 179        │

>│  dictionary : 0/0, 0/0, 4/42.5k                     │  imported : n/a        │

>│       havoc : 151/2.40M, 0/0                        │  variable : 0          │

>│        trim : 52.20%/10.7k, 14.91%                  ├────────────────────────┘

>^C────────────────────────────────────────────────────┘             [cpu: 38%]


So at first I thought I could only work with secret keys, but it turns out for each terminal we have to run

> **_export LD_LIBRARY_PATH="/home/willy/testing/usr/local/lib"_**

in order to get LibreSSL working correctly

At the moment, Thursday July 2nd, 2015 at 11:33pm CST, I am rerunning LibreSSL on keys, but also on certificates and csr files. Considering they website I'm following claims they've run it for two to three days we'll see if I can find anything interesting in a few hours. I'm considering creating some cloud instance in order to outsource the computation.

I think I will continue with LibreSSL. I will try to fuzz as much as I can and compare that to the change logs for each version.